In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pyodbc as pyo
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from random import randint
import random
from bs4 import Comment
import re

In [2]:
%run USER_PASSWORD.ipynb
host_name = ''
if user_name in users_list:
    host_name = LOCAL[user_name]
    print(f"is your local: {host_name}")
else:
    host_name = input("Tên không tìm thấy. Vui lòng tự điền Local: ").strip()
    print(f"Local bạn đã nhập là: {host_name}")

Please input your user name: quynhnt3
----------------------------------------------------
Please input your pass word: 28082001
HI 🎉✨🎁🎈
Quỳnh 🎀
WELCOME TO THE WORLD OF DATA
is your local: LT32046SME200


In [ ]:
choice_dict = {
    1: "Đang mở thầu",
    2: "Đang xét thầu",
    3: "Đã trúng thầu",
    4: "All"
}

print('               What do you want to crawl?? \n 1.--->   Đang mở thầu \n 2.--->   Đang xét thầu \n 3.--->   Đã trúng thầu \n 4.--->   All')
Loai_goi_thau = None
while True:
    try:
        choice = int(input('Please Enter Your Choice (1-4): '))
        if choice in choice_dict:
            Loai_goi_thau = choice_dict[choice]
            break
        else:
            print('INCORRECT!!!! Your choice is invalid, please make it again!!')
    except ValueError:
        print('INCORRECT!!!! Please enter a valid number between 1 and 4.')

print('   ------------------------------------------------->     CORRECT!!!! YOUR CHOICE IS', Loai_goi_thau)
print('   ')

internet_connection = input('Which internet connection do you want to access (WIFI or LAN): ').strip()
print('   ------->     YOUR CHOICE IS:', internet_connection, '  Please make sure you are using the correct internet connection')
print('   ')

ready = input('Are you ready to run (Yes/No): ').strip()



               What do you want to crawl?? 
 1.--->   Đang mở thầu 
 2.--->   Đang xét thầu 
 3.--->   Đã trúng thầu 
 4.--->   All
Please Enter Your Choice (1-4): 3
   ------------------------------------------------->     CORRECT!!!! YOUR CHOICE IS Đã trúng thầu
   


In [ ]:

try:
    # Kết nối với cơ sở dữ liệu SQL Server
    connection_3 = pyo.connect(
        driver='{ODBC Driver 17 for SQL Server}',
        host=host_name,
        database='SME',
        trusted_connection='yes'
    )
    
    # Tạo đối tượng cursor để thực hiện truy vấn
    cursor_3 = connection_3.cursor()
    
    # Viết truy vấn SQL với tham số cho biến `Loai_goi_thau`
    query = """
    SELECT * 
    FROM SME..TAIMX_DIM_DU_LIEU_GOI_THAU_OVERVIEW 
    WHERE Trang_thai_goi_thau = ? AND Remark = 0
    """
    
    # Thực hiện truy vấn với tham số truyền vào
    query_result = pd.read_sql_query(query, connection_3, params=[Loai_goi_thau])
    
    # Hiển thị kết quả
    #print(query_result)
    
except Exception as e:
    print("Lỗi khi kết nối hoặc truy vấn cơ sở dữ liệu:", e)
finally:
    # Đóng kết nối sau khi hoàn tất
    if 'connection_3' in locals() and connection_3:
        connection_3.close()


In [ ]:
# connection_3 = pyo.connect(driver='{ODBC Driver 17 for SQL Server}'
#                         ,host= 'LT32046SME200'
#                         ,database='SME'
#                         ,trusted_connection='yes')
# cursor_3= connection_3.cursor()
# query = "SELECT * FROM SME..TAIMX_DIM_DU_LIEU_GOI_THAU_OVERVIEW_1 where Trang_thai_goi_thau =N'Đang xét thầu' and Remark = 0"
# query_result = pd.read_sql(sql=query,con=connection_3)

In [ ]:
#  len(query_result)

In [ ]:
# link = query_result.iloc[0,3]
# print(link)

In [ ]:
def Da_trung_thau():
    connection_3 = pyo.connect(driver='{ODBC Driver 17 for SQL Server}'
                        ,host= host_name
                        ,database='SME'
                        ,trusted_connection='yes')
    cursor_3= connection_3.cursor()
    #WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.ID, 'info-general')))
    time.sleep(randint(10,20))
    #WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '//*[@id="tender-notice"]/div[2]/div/div[1]/div[1]/div[2]/ul/li/a')))
    page_source = BeautifulSoup(browser.page_source,'html.parser')

    # Lấy các thông tin chung từ thông báo mời thầu
    general_infor = page_source.find('div',id='info-general')
         #Thông tin cơ bản
    thongtincoban = general_infor.contents[4]
    thongtincoban_title = thongtincoban.find('div',class_='infomation__content__title')
    Ma_TBMT = thongtincoban_title.find_next_sibling('div').text.strip()
         #Thông tin gói thầu
    thongtingoithau = general_infor.contents[8]
    thongtingoithau_title = thongtingoithau.find_all('div',class_='infomation__content__title')
    pre_key_1 = []
    pre_value_1 = []
    for tt_goithau in thongtingoithau_title:
        key_1 = tt_goithau.text.strip()
        try:
            tt_goi_thau_value = tt_goithau.find_next_sibling('div')
            value_1 = tt_goi_thau_value.text.strip()
        except (AttributeError, NameError):
            value_1 = 'No info'
        pre_key_1.append(key_1)
        pre_value_1.append(value_1)
    ttgoithau_dict = dict(zip(pre_key_1,pre_value_1))
    ttgoithau_list = ['Tên gói thầu','Bên mời thầu','Chi tiết nguồn vốn','Lĩnh vực','Hình thức lựa chọn nhà thầu','Loại hợp đồng'
                     ,'Trong nước/ Quốc tế','Phương thức lựa chọn nhà thầu','Thời gian thực hiện gói thầu']
    pre_1 = []
    for goithau_item in ttgoithau_list:
        if goithau_item in pre_key_1:
            pre_1.append(ttgoithau_dict[goithau_item])
        else:
            pre_1.append('No info')
    Ten_goi_thau = pre_1[0]
    Ben_moi_thau = pre_1[1]
    Nguon_ngan_sach = pre_1[2]
    Linh_vuc = pre_1[3]
    Hinh_thuc_chon_thau = pre_1[4]
    Loai_hop_dong = pre_1[5]
    Trongnuoc_quocte = pre_1[6]
    Phuong_thuc_chon_thau = pre_1[7]
    Thoi_gian_THHD = pre_1[8]
         #Thông tin dự thầu
    thongtingoithau = general_infor.contents[14]
    
    # Tìm tất cả các thẻ `div` có class `infomation__content__title`
    thongtingoithau_title = thongtingoithau.find_all('div', class_='infomation__content__title')
    
    # Khởi tạo danh sách để chứa tiêu đề và giá trị
    pre_key_2 = []
    pre_value_2 = []
    
    # Duyệt qua từng tiêu đề và lấy nội dung của nó cùng với giá trị tương ứng
    for tt_duthau in thongtingoithau_title:
        key_2 = tt_duthau.text.strip()  # Tiêu đề (key)
        
        # Tìm giá trị tương ứng sau thẻ `div` tiêu đề
        try:
            tt_duthau_value = tt_duthau.find_next_sibling('div')
            value_2 = tt_duthau_value.text.strip() if tt_duthau_value else 'No info'
        except (AttributeError, NameError):
            value_2 = 'No info'
        
        # Thêm tiêu đề và giá trị vào danh sách
        pre_key_2.append(key_2)
        pre_value_2.append(value_2)
    
    # Tạo từ điển từ hai danh sách
    ttduthau_dict = dict(zip(pre_key_2, pre_value_2))
    
    # Lấy thông tin "Số tiền bảo đảm dự thầu" và giữ lại định dạng số và đơn vị "VND"
    So_tien_dam_bao_du_thau_raw = ttduthau_dict.get('Số tiền bảo đảm dự thầu', 'No info')
    match = re.search(r'\d{1,3}(?:\.\d{3})* VND', So_tien_dam_bao_du_thau_raw)  # Tìm theo định dạng "x.xxx.xxx VND"
    So_tien_dam_bao_du_thau = match.group() if match else 'No info'
    
    # Lấy "Hình thức đảm bảo dự thầu" và chỉ lấy phần trước dấu "("
    Hinh_thuc_dam_bao_du_thau_raw = ttduthau_dict.get('Hình thức đảm bảo dự thầu', 'No info').strip()
    Hinh_thuc_dam_bao_du_thau = Hinh_thuc_dam_bao_du_thau_raw.split('(')[0].strip()

    # Loại công trình
    Loai_cong_trinh = ttduthau_dict.get('Loại công trình', 'No info').strip()
    # Thông tin quyêt định phê duyệt
    quyetdinhpheduyet = general_infor.contents[16]
    
    # Tìm tất cả các thẻ `div` có class `infomation__content__title`
    quyetdinhpheduyet_tile = quyetdinhpheduyet.find_all('div', class_='infomation__content__title')
    
    # Khởi tạo danh sách để chứa tiêu đề và giá trị
    pre_key_3 = []
    pre_value_3 = []
    
    # Duyệt qua từng tiêu đề và lấy nội dung của nó cùng với giá trị tương ứng
    for tt_quyetdinh in quyetdinhpheduyet_tile:
        key_3 = tt_quyetdinh.text.strip()  # Tiêu đề (key)
        
        # Tìm giá trị tương ứng sau thẻ `div` tiêu đề
        try:
            tt_quyetdinh_value = tt_quyetdinh.find_next_sibling('div')
            value_3 = tt_quyetdinh_value.text.strip() if tt_quyetdinh_value else 'No info'
        except (AttributeError, NameError):
            value_3 = 'No info'
        
        # Thêm tiêu đề và giá trị vào danh sách
        pre_key_3.append(key_3)
        pre_value_3.append(value_3)
    
    # Tạo từ điển từ hai danh sách
    ttquyetdinh_dict = dict(zip(pre_key_3, pre_value_3))
    
    # Danh sách các thông tin cần tìm
    ttquyetdinh_list = ['Số quyết định phê duyệt', 'Ngày phê duyệt', 'Cơ quan ban hành quyết định']
    pre_3 = []
    
    # Kiểm tra từng mục trong `ttquyetdinh_list` có tồn tại trong `pre_key_3` không
    for quyetdinh_item in ttquyetdinh_list:
        if quyetdinh_item in pre_key_3:
            pre_3.append(ttquyetdinh_dict[quyetdinh_item])
        else:
            pre_3.append('No info')
    
    # Gán giá trị cho các biến cuối cùng
    So_QĐPD = pre_3[0].strip()
    Ngay_ky_QĐPD = pre_3[1].strip()
    Co_quan_ban_hanh = pre_3[2].strip()


    # Lấy thông tin từ biên bản mời thầu:
    chuyen_1 = browser.find_element(By.XPATH,'//*[@id="tender-notice"]/div/div/div[2]/div/div[1]/div[1]/div[2]/ul/li[2]')
    chuyen_1.click()
    time.sleep(randint(10,20))
    page_source = BeautifulSoup(browser.page_source,'html.parser')
    bidOpeningMinutes = page_source.find('div',id='bidOpeningMinutes')
    info_table = bidOpeningMinutes.find('div',class_='card border--none')
    info_titles = info_table.find_all('div',class_='infomation__content__title')
    infor_pre_key = []
    infor_pre_value = []
    for info_title in info_titles:
        infor_key = info_title.text.strip()
        try:
            inf = info_title.find_next_sibling('div')
            infor_value = inf.text.strip()
        except (AttributeError, NameError):
            infor_value = 'No info'
        infor_pre_key.append(infor_key)
        infor_pre_value.append(infor_value)
    info_dict = dict(zip(infor_pre_key,infor_pre_value))
    info_list = ['Giá gói thầu','Tổng số nhà thầu tham dự']
    info_pre = []
    for info_item in info_list:
        if info_item in infor_pre_key:
            info_pre.append(info_dict[info_item])
        else:
            info_pre.append('No info')
    Gia_goi_thau = info_pre[0].replace(".","")
    So_DN_du_thau = info_pre[1]
    table = bidOpeningMinutes.find('table',class_='table table-expand')
    table_td_tags = table.find_all('td')
    row_pre = []
    #tai_test_1 = []
    for td_tag in table_td_tags:
        td_text = td_tag.text.strip()
        #tai_test_1.append(td_text)
        if td_text.startswith('vn')==True:
            row_pre.append(td_text)
    while len(row_pre) < 20:
        row_pre.append('X')
    DN_du_thau_01 = row_pre[0].strip()
    DN_du_thau_02 = row_pre[1].strip()
    DN_du_thau_03 = row_pre[2].strip()
    DN_du_thau_04 = row_pre[3].strip()
    DN_du_thau_05 = row_pre[4].strip()
    DN_du_thau_06 = row_pre[5].strip()
    DN_du_thau_07 = row_pre[6].strip()
    DN_du_thau_08 = row_pre[7].strip()
    DN_du_thau_09 = row_pre[8].strip()
    DN_du_thau_10 = row_pre[9].strip()
    DN_du_thau_11 = row_pre[10].strip()
    DN_du_thau_12 = row_pre[11].strip()
    DN_du_thau_13 = row_pre[12].strip()
    DN_du_thau_14 = row_pre[13].strip()
    DN_du_thau_15 = row_pre[14].strip()
    DN_du_thau_16 = row_pre[15].strip()
    DN_du_thau_17 = row_pre[16].strip()
    DN_du_thau_18 = row_pre[17].strip()
    DN_du_thau_19 = row_pre[18].strip()
    DN_du_thau_20 = row_pre[19].strip()



    #Lấy thông tin kết quả lựa chọn nhà thầu
    try:
        chuyen_2 = browser.find_element(By.XPATH,'//*[@id="tender-notice"]/div/div/div[2]/div/div[1]/div[1]/div[2]/ul/li[5]/a')
    except:
        chuyen_2 = browser.find_element(By.XPATH,'//*[@id="tender-notice"]/div/div/div[2]/div/div[1]/div[1]/div[2]/ul/li[3]/a')
    chuyen_2.click()
    time.sleep(randint(10,20))
    page_source = BeautifulSoup(browser.page_source,'html.parser')   
    contractorSelectionResults = page_source.find('div',id='contractorSelectionResults')
    KQ_table = contractorSelectionResults.find('table',class_='table table-notStt table-expand')
    KQ_pre = []
    KQ_table_td_tags = KQ_table.find_all('td')
    for KQ_td_tag in KQ_table_td_tags:
        KQ_td_text = KQ_td_tag.text.strip()
        if KQ_td_text.startswith('vn')==True:
            KQ_pre.append(KQ_td_text)
    while len(KQ_pre) < 20:
          KQ_pre.append('X')
    DN_trung_thau_01 = KQ_pre[0]
    DN_trung_thau_02 = KQ_pre[1]
    DN_trung_thau_03 = KQ_pre[2]
    DN_trung_thau_04 = KQ_pre[3]
    DN_trung_thau_05 = KQ_pre[4]
    DN_trung_thau_06 = KQ_pre[5]
    DN_trung_thau_07 = KQ_pre[6]
    DN_trung_thau_08 = KQ_pre[7]
    DN_trung_thau_09 = KQ_pre[8]
    DN_trung_thau_10 = KQ_pre[9]
    DN_trung_thau_11 = KQ_pre[10]
    DN_trung_thau_12 = KQ_pre[11]
    DN_trung_thau_13 = KQ_pre[12]
    DN_trung_thau_14 = KQ_pre[13]
    DN_trung_thau_15 = KQ_pre[14]
    DN_trung_thau_16 = KQ_pre[15]
    DN_trung_thau_17 = KQ_pre[16]
    DN_trung_thau_18 = KQ_pre[17]
    DN_trung_thau_19 = KQ_pre[18]
    DN_trung_thau_20 = KQ_pre[19]
    td_tags_pre = []
    td_tags = contractorSelectionResults.find_all('td') 
    for td_tag in td_tags:
        td_tags_pre.append(td_tag.text.strip())
    try:
        Ngay_ky_HD = td_tags_pre[-1]
        Ngay_ky_HD = datetime.strptime(Ngay_ky_HD, '%d/%m/%Y')
    except ValueError as ve:
        Ngay_ky_HD = '01/01/2999'
        Ngay_ky_HD = datetime.strptime(Ngay_ky_HD, '%d/%m/%Y')
    Trang_thai_goi_thau = 'Đã trúng thầu'
    now = datetime.now()
    Ngay_keo_du_lieu = now.strftime("%d/%m/%Y %H:%M:%S")
    Ngay_keo_du_lieu = datetime.strptime(Ngay_keo_du_lieu, "%d/%m/%Y %H:%M:%S")
    Sql_2 = "Insert into SME..TAIMX_SRC_Du_LIEU_GOI_THAU_DETAIL (Ma_TBMT,Ten_goi_thau,Ben_moi_thau,Nguon_ngan_sach,Linh_vuc,Hinh_thuc_chon_thau,Loai_hop_dong,Trongnuoc_quocte,Phuong_thuc_chon_thau,Thoi_gian_THHD,So_tien_dam_bao_du_thau,Hinh_thuc_dam_bao_du_thau,Loai_cong_trinh,So_QĐPD,Ngay_ky_QĐPD,Co_quan_ban_hanh,Gia_goi_thau,So_DN_du_thau,DN_du_thau_01,DN_du_thau_02,DN_du_thau_03,DN_du_thau_04,DN_du_thau_05,DN_du_thau_06,DN_du_thau_07,DN_du_thau_08,DN_du_thau_09,DN_du_thau_10,DN_du_thau_11,DN_du_thau_12,DN_du_thau_13,DN_du_thau_14,DN_du_thau_15,DN_du_thau_16,DN_du_thau_17,DN_du_thau_18,DN_du_thau_19,DN_du_thau_20,DN_trung_thau_01,DN_trung_thau_02,DN_trung_thau_03,DN_trung_thau_04,DN_trung_thau_05,DN_trung_thau_06,DN_trung_thau_07,DN_trung_thau_08,DN_trung_thau_09,DN_trung_thau_10,DN_trung_thau_11,DN_trung_thau_12,DN_trung_thau_13,DN_trung_thau_14,DN_trung_thau_15,DN_trung_thau_16,DN_trung_thau_17,DN_trung_thau_18,DN_trung_thau_19,DN_trung_thau_20,Ngay_ky_HD,Trang_thai_goi_thau,Ngay_keo_du_lieu) Values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
    cursor_3.execute(Sql_2,(Ma_TBMT,Ten_goi_thau,Ben_moi_thau,Nguon_ngan_sach,Linh_vuc,Hinh_thuc_chon_thau,Loai_hop_dong,Trongnuoc_quocte,Phuong_thuc_chon_thau,Thoi_gian_THHD,So_tien_dam_bao_du_thau,Hinh_thuc_dam_bao_du_thau,Loai_cong_trinh,So_QĐPD,Ngay_ky_QĐPD,Co_quan_ban_hanh,Gia_goi_thau,So_DN_du_thau,DN_du_thau_01,DN_du_thau_02,DN_du_thau_03,DN_du_thau_04,DN_du_thau_05,DN_du_thau_06,DN_du_thau_07,DN_du_thau_08,DN_du_thau_09,DN_du_thau_10,DN_du_thau_11,DN_du_thau_12,DN_du_thau_13,DN_du_thau_14,DN_du_thau_15,DN_du_thau_16,DN_du_thau_17,DN_du_thau_18,DN_du_thau_19,DN_du_thau_20,DN_trung_thau_01,DN_trung_thau_02,DN_trung_thau_03,DN_trung_thau_04,DN_trung_thau_05,DN_trung_thau_06,DN_trung_thau_07,DN_trung_thau_08,DN_trung_thau_09,DN_trung_thau_10,DN_trung_thau_11,DN_trung_thau_12,DN_trung_thau_13,DN_trung_thau_14,DN_trung_thau_15,DN_trung_thau_16,DN_trung_thau_17,DN_trung_thau_18,DN_trung_thau_19,DN_trung_thau_20,Ngay_ky_HD,Trang_thai_goi_thau,Ngay_keo_du_lieu))
    connection_3.commit()
    Sql_update_sentence = "Update SME..TAIMX_DIM_DU_LIEU_GOI_THAU_OVERVIEW Set Remark = '1' where Ma_TBMT ='"+ Ma_TBMT +"'"
    cursor_3.execute(Sql_update_sentence)
    connection_3.commit()
    print('Written successfully for the contract no:',Ma_TBMT)
    print('-----------------------------------', j)
    browser.close()
    time.sleep(randint(1,3))


In [ ]:
def Dang_xet_thau():
    connection_3 = pyo.connect(driver='{ODBC Driver 17 for SQL Server}'
                        ,host= host_name
                        ,database='SME'
                        ,trusted_connection='yes')
    cursor_3= connection_3.cursor()
    WebDriverWait(browser, 60).until(EC.presence_of_element_located((By.ID, 'info-general')))
    page_source = BeautifulSoup(browser.page_source,'html.parser')

    # Lấy các thông tin chung từ thông báo mời thầu

    general_infor = page_source.find('div',id='info-general')

         #Thông tin cơ bản
    thongtincoban = general_infor.contents[4]
    thongtincoban_title = thongtincoban.find('div',class_='infomation__content__title')
    Ma_TBMT = thongtincoban_title.find_next_sibling('div').text.strip()

         #Thông tin gói thầu

    thongtingoithau = general_infor.contents[8]
    
    # Tìm tất cả các thẻ `div` có class `infomation__content__title`
    thongtingoithau_title = thongtingoithau.find_all('div', class_='infomation__content__title')
    
    # Khởi tạo danh sách để chứa tiêu đề và giá trị
    pre_key_1 = []
    pre_value_1 = []
    
    # Duyệt qua từng tiêu đề và lấy nội dung của nó cùng với giá trị tương ứng
    for tt_goithau in thongtingoithau_title:
        key_1 = tt_goithau.text.strip()
        
        # Tìm giá trị tương ứng sau thẻ `div` tiêu đề
        try:
            tt_goi_thau_value = tt_goithau.find_next_sibling('div')
            value_1 = tt_goi_thau_value.text.strip() if tt_goi_thau_value else 'No info'
        except (AttributeError, NameError):
            value_1 = 'No info'
        
        # Thêm tiêu đề và giá trị vào danh sách
        pre_key_1.append(key_1)
        pre_value_1.append(value_1)
    
    # Tạo từ điển từ hai danh sách
    ttgoithau_dict = dict(zip(pre_key_1, pre_value_1))
    #print(ttgoithau_dict)
    #Danh sách các thông tin cần tìm
    ttgoithau_list = [
        'Tên gói thầu', 'Bên mời thầu', 'Chi tiết nguồn vốn', 'Lĩnh vực', 
        'Hình thức lựa chọn nhà thầu', 'Loại hợp đồng', 'Trong nước/ Quốc tế', 
        'Phương thức lựa chọn nhà thầu', 'Thời gian thực hiện gói thầu'
    ]
    pre_1 = []
    
    # Kiểm tra từng mục trong `ttgoithau_list` có tồn tại trong `pre_key_1` không
    for goithau_item in ttgoithau_list:
        pre_1.append(ttgoithau_dict.get(goithau_item, 'No info'))
    
    # Gán giá trị cho các biến cuối cùng
    Ten_goi_thau = pre_1[0]
    Ben_moi_thau = pre_1[1]
    Nguon_ngan_sach = pre_1[2]
    Linh_vuc = pre_1[3]
    Hinh_thuc_chon_thau = pre_1[4]
    Loai_hop_dong = pre_1[5]
    Trongnuoc_quocte = pre_1[6]
    Phuong_thuc_chon_thau = pre_1[7]
    Thoi_gian_THHD = pre_1[8]


         #Thông tin dự thầu
    thongtingoithau = general_infor.contents[14]
    
    # Tìm tất cả các thẻ `div` có class `infomation__content__title`
    thongtingoithau_title = thongtingoithau.find_all('div', class_='infomation__content__title')
    
    # Khởi tạo danh sách để chứa tiêu đề và giá trị
    pre_key_2 = []
    pre_value_2 = []
    
    # Duyệt qua từng tiêu đề và lấy nội dung của nó cùng với giá trị tương ứng
    for tt_duthau in thongtingoithau_title:
        key_2 = tt_duthau.text.strip()  # Tiêu đề (key)
        
        # Tìm giá trị tương ứng sau thẻ `div` tiêu đề
        try:
            tt_duthau_value = tt_duthau.find_next_sibling('div')
            value_2 = tt_duthau_value.text.strip() if tt_duthau_value else 'No info'
        except (AttributeError, NameError):
            value_2 = 'No info'
        
        # Thêm tiêu đề và giá trị vào danh sách
        pre_key_2.append(key_2)
        pre_value_2.append(value_2)
    
    # Tạo từ điển từ hai danh sách
    ttduthau_dict = dict(zip(pre_key_2, pre_value_2))
    
    # Lấy thông tin "Số tiền bảo đảm dự thầu" và giữ lại định dạng số và đơn vị "VND"
    So_tien_dam_bao_du_thau_raw = ttduthau_dict.get('Số tiền bảo đảm dự thầu', 'No info')
    match = re.search(r'\d{1,3}(?:\.\d{3})* VND', So_tien_dam_bao_du_thau_raw)  # Tìm theo định dạng "x.xxx.xxx VND"
    So_tien_dam_bao_du_thau = match.group() if match else 'No info'
    
    # Lấy "Hình thức đảm bảo dự thầu" và chỉ lấy phần trước dấu "("
    Hinh_thuc_dam_bao_du_thau_raw = ttduthau_dict.get('Hình thức đảm bảo dự thầu', 'No info').strip()
    Hinh_thuc_dam_bao_du_thau = Hinh_thuc_dam_bao_du_thau_raw.split('(')[0].strip()

    # Loại công trình
    Loai_cong_trinh = ttduthau_dict.get('Loại công trình', 'No info').strip()
    # Thông tin quyêt định phê duyệt
    quyetdinhpheduyet = general_infor.contents[16]
    
    # Tìm tất cả các thẻ `div` có class `infomation__content__title`
    quyetdinhpheduyet_tile = quyetdinhpheduyet.find_all('div', class_='infomation__content__title')
    
    # Khởi tạo danh sách để chứa tiêu đề và giá trị
    pre_key_3 = []
    pre_value_3 = []
    
    # Duyệt qua từng tiêu đề và lấy nội dung của nó cùng với giá trị tương ứng
    for tt_quyetdinh in quyetdinhpheduyet_tile:
        key_3 = tt_quyetdinh.text.strip()  # Tiêu đề (key)
        
        # Tìm giá trị tương ứng sau thẻ `div` tiêu đề
        try:
            tt_quyetdinh_value = tt_quyetdinh.find_next_sibling('div')
            value_3 = tt_quyetdinh_value.text.strip() if tt_quyetdinh_value else 'No info'
        except (AttributeError, NameError):
            value_3 = 'No info'
        
        # Thêm tiêu đề và giá trị vào danh sách
        pre_key_3.append(key_3)
        pre_value_3.append(value_3)
    
    # Tạo từ điển từ hai danh sách
    ttquyetdinh_dict = dict(zip(pre_key_3, pre_value_3))
    
    # Danh sách các thông tin cần tìm
    ttquyetdinh_list = ['Số quyết định phê duyệt', 'Ngày phê duyệt', 'Cơ quan ban hành quyết định']
    pre_3 = []
    
    # Kiểm tra từng mục trong `ttquyetdinh_list` có tồn tại trong `pre_key_3` không
    for quyetdinh_item in ttquyetdinh_list:
        if quyetdinh_item in pre_key_3:
            pre_3.append(ttquyetdinh_dict[quyetdinh_item])
        else:
            pre_3.append('No info')
    
    # Gán giá trị cho các biến cuối cùng
    So_QĐPD = pre_3[0].strip()
    Ngay_ky_QĐPD = pre_3[1].strip()
    Co_quan_ban_hanh = pre_3[2].strip()
    # Lấy thông tin từ biên bản mời thầu:

    try:
        chuyen_1 = browser.find_element(By.XPATH, '//*[@id="tender-notice"]/div/div/div[2]/div/div[1]/div[1]/div[2]/ul/li[2]')
        chuyen_1.click()
        time.sleep(randint(10, 20))  # Chờ đợi một khoảng thời gian ngẫu nhiên
    except Exception as e:
        print("Không thể chuyển tab:", e)

    # Phân tích trang hiện tại bằng BeautifulSoup
    page_source = BeautifulSoup(browser.page_source, 'html.parser')

    # Tìm phần tử có id 'bidOpeningMinutes' để lấy thông tin đấu thầu
    bid_opening_minutes = page_source.find('div', id='bidOpeningMinutes')
    if bid_opening_minutes:
        info_table = bid_opening_minutes.find('div', class_='card border--none')

        # Tìm tất cả các tiêu đề và giá trị trong `info_table`
        info_titles = info_table.find_all('div', class_='infomation__content__title')
        infor_pre_key = []
        infor_pre_value = []

        for info_title in info_titles:
            infor_key = info_title.text.strip()
            try:
                # Lấy giá trị tiếp theo sau tiêu đề
                inf = info_title.find_next_sibling('div')
                infor_value = inf.text.strip() if inf else 'No info'
            except (AttributeError, NameError):
                infor_value = 'No info'

            infor_pre_key.append(infor_key)
            infor_pre_value.append(infor_value)

        # Tạo từ điển với cặp tiêu đề và giá trị
        info_dict = dict(zip(infor_pre_key, infor_pre_value))

        # Lấy các thông tin cụ thể
        info_list = ['Giá gói thầu', 'Tổng số nhà thầu tham dự']
        info_pre = [info_dict.get(info_item, 'No info') for info_item in info_list]

        Gia_goi_thau = info_pre[0].replace(".", "")  # Loại bỏ dấu chấm trong giá trị
        So_DN_du_thau = info_pre[1]
    table = bid_opening_minutes.find('table',class_='table table-expand')
    table_td_tags = table.find_all('td')
    row_pre = []
    #tai_test_1 = []
    for td_tag in table_td_tags:
        td_text = td_tag.text.strip()
        #tai_test_1.append(td_text)
        if td_text.startswith('vn')==True:
            row_pre.append(td_text)
    while len(row_pre) < 20:
        row_pre.append('X')
    DN_du_thau_01 = row_pre[0].strip()
    DN_du_thau_02 = row_pre[1].strip()
    DN_du_thau_03 = row_pre[2].strip()
    DN_du_thau_04 = row_pre[3].strip()
    DN_du_thau_05 = row_pre[4].strip()
    DN_du_thau_06 = row_pre[5].strip()
    DN_du_thau_07 = row_pre[6].strip()
    DN_du_thau_08 = row_pre[7].strip()
    DN_du_thau_09 = row_pre[8].strip()
    DN_du_thau_10 = row_pre[9].strip()
    DN_du_thau_11 = row_pre[10].strip()
    DN_du_thau_12 = row_pre[11].strip()
    DN_du_thau_13 = row_pre[12].strip()
    DN_du_thau_14 = row_pre[13].strip()
    DN_du_thau_15 = row_pre[14].strip()
    DN_du_thau_16 = row_pre[15].strip()
    DN_du_thau_17 = row_pre[16].strip()
    DN_du_thau_18 = row_pre[17].strip()
    DN_du_thau_19 = row_pre[18].strip()
    DN_du_thau_20 = row_pre[19].strip()


    #Lấy thông tin kết quả lựa chọn nhà thầu
    DN_trung_thau_01 = 'Đang xét thầu'
    DN_trung_thau_02 = 'Đang xét thầu'
    DN_trung_thau_03 = 'Đang xét thầu'
    DN_trung_thau_04 = 'Đang xét thầu'
    DN_trung_thau_05 = 'Đang xét thầu'
    DN_trung_thau_06 = 'Đang xét thầu'
    DN_trung_thau_07 = 'Đang xét thầu'
    DN_trung_thau_08 = 'Đang xét thầu'
    DN_trung_thau_09 = 'Đang xét thầu'
    DN_trung_thau_10 = 'Đang xét thầu'
    DN_trung_thau_11 = 'Đang xét thầu'
    DN_trung_thau_12 = 'Đang xét thầu'
    DN_trung_thau_13 = 'Đang xét thầu'
    DN_trung_thau_14 = 'Đang xét thầu'
    DN_trung_thau_15 = 'Đang xét thầu'
    DN_trung_thau_16 = 'Đang xét thầu'
    DN_trung_thau_17 = 'Đang xét thầu'
    DN_trung_thau_18 = 'Đang xét thầu'
    DN_trung_thau_19 = 'Đang xét thầu'
    DN_trung_thau_20 = 'Đang xét thầu'
    Ngay_ky_HD = '01/01/2999'
    Ngay_ky_HD = datetime.strptime(Ngay_ky_HD, '%d/%m/%Y')
    Trang_thai_goi_thau = 'Đang xét thầu'
    now = datetime.now()
    Ngay_keo_du_lieu = now.strftime("%d/%m/%Y %H:%M:%S")
    Ngay_keo_du_lieu = datetime.strptime(Ngay_keo_du_lieu, "%d/%m/%Y %H:%M:%S")
    Sql_2 = "Insert into SME..TAIMX_SRC_Du_LIEU_GOI_THAU_DETAIL (Ma_TBMT,Ten_goi_thau,Ben_moi_thau,Nguon_ngan_sach,Linh_vuc,Hinh_thuc_chon_thau,Loai_hop_dong,Trongnuoc_quocte,Phuong_thuc_chon_thau,Thoi_gian_THHD,So_tien_dam_bao_du_thau,Hinh_thuc_dam_bao_du_thau,Loai_cong_trinh,So_QĐPD,Ngay_ky_QĐPD,Co_quan_ban_hanh,Gia_goi_thau,So_DN_du_thau,DN_du_thau_01,DN_du_thau_02,DN_du_thau_03,DN_du_thau_04,DN_du_thau_05,DN_du_thau_06,DN_du_thau_07,DN_du_thau_08,DN_du_thau_09,DN_du_thau_10,DN_du_thau_11,DN_du_thau_12,DN_du_thau_13,DN_du_thau_14,DN_du_thau_15,DN_du_thau_16,DN_du_thau_17,DN_du_thau_18,DN_du_thau_19,DN_du_thau_20,DN_trung_thau_01,DN_trung_thau_02,DN_trung_thau_03,DN_trung_thau_04,DN_trung_thau_05,DN_trung_thau_06,DN_trung_thau_07,DN_trung_thau_08,DN_trung_thau_09,DN_trung_thau_10,DN_trung_thau_11,DN_trung_thau_12,DN_trung_thau_13,DN_trung_thau_14,DN_trung_thau_15,DN_trung_thau_16,DN_trung_thau_17,DN_trung_thau_18,DN_trung_thau_19,DN_trung_thau_20,Ngay_ky_HD,Trang_thai_goi_thau,Ngay_keo_du_lieu) Values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
    cursor_3.execute(Sql_2,(Ma_TBMT,Ten_goi_thau,Ben_moi_thau,Nguon_ngan_sach,Linh_vuc,Hinh_thuc_chon_thau,Loai_hop_dong,Trongnuoc_quocte,Phuong_thuc_chon_thau,Thoi_gian_THHD,So_tien_dam_bao_du_thau,Hinh_thuc_dam_bao_du_thau,Loai_cong_trinh,So_QĐPD,Ngay_ky_QĐPD,Co_quan_ban_hanh,Gia_goi_thau,So_DN_du_thau,DN_du_thau_01,DN_du_thau_02,DN_du_thau_03,DN_du_thau_04,DN_du_thau_05,DN_du_thau_06,DN_du_thau_07,DN_du_thau_08,DN_du_thau_09,DN_du_thau_10,DN_du_thau_11,DN_du_thau_12,DN_du_thau_13,DN_du_thau_14,DN_du_thau_15,DN_du_thau_16,DN_du_thau_17,DN_du_thau_18,DN_du_thau_19,DN_du_thau_20,DN_trung_thau_01,DN_trung_thau_02,DN_trung_thau_03,DN_trung_thau_04,DN_trung_thau_05,DN_trung_thau_06,DN_trung_thau_07,DN_trung_thau_08,DN_trung_thau_09,DN_trung_thau_10,DN_trung_thau_11,DN_trung_thau_12,DN_trung_thau_13,DN_trung_thau_14,DN_trung_thau_15,DN_trung_thau_16,DN_trung_thau_17,DN_trung_thau_18,DN_trung_thau_19,DN_trung_thau_20,Ngay_ky_HD,Trang_thai_goi_thau,Ngay_keo_du_lieu))
    connection_3.commit()
    Sql_update_sentence = "Update SME..TAIMX_DIM_DU_LIEU_GOI_THAU_OVERVIEW Set Remark = '1' where Ma_TBMT ='"+ Ma_TBMT +"'"
    cursor_3.execute(Sql_update_sentence)
    connection_3.commit()
    print('Written successfully for the contract no:',Ma_TBMT)
    print('-----------------------------------', j)
    browser.close()

In [ ]:
def Dang_mo_thau():
    connection_3 = pyo.connect(driver='{ODBC Driver 17 for SQL Server}'
                        ,host= host_name
                        ,database='SME'
                        ,trusted_connection='yes')
    cursor_3= connection_3.cursor()    
    WebDriverWait(browser, 60).until(EC.presence_of_element_located((By.ID, 'info-general')))
    #WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '//*[@id="tender-notice"]/div[2]/div/div[1]/div[1]/div[2]/ul/li/a')))
    page_source = BeautifulSoup(browser.page_source,'html.parser')

    # Lấy các thông tin chung từ thông báo mời thầu
    general_infor = page_source.find('div',id='info-general')
         #Thông tin cơ bản
    thongtincoban = general_infor.contents[4]
    thongtincoban_title = thongtincoban.find('div',class_='infomation__content__title')
    Ma_TBMT = thongtincoban_title.find_next_sibling('div').text.strip()
         #Thông tin gói thầu
    thongtingoithau = general_infor.contents[8]
    thongtingoithau_title = thongtingoithau.find_all('div',class_='infomation__content__title')
    pre_key_1 = []
    pre_value_1 = []
    for tt_goithau in thongtingoithau_title:
        key_1 = tt_goithau.text.strip()
        try:
            tt_goi_thau_value = tt_goithau.find_next_sibling('div')
            value_1 = tt_goi_thau_value.text.strip()
        except (AttributeError, NameError):
            value_1 = 'No info'
        pre_key_1.append(key_1)
        pre_value_1.append(value_1)
    ttgoithau_dict = dict(zip(pre_key_1,pre_value_1))
    ttgoithau_list = [
        'Tên gói thầu', 'Bên mời thầu', 'Chi tiết nguồn vốn', 'Lĩnh vực', 
        'Hình thức lựa chọn nhà thầu', 'Loại hợp đồng', 'Trong nước/ Quốc tế', 
        'Phương thức lựa chọn nhà thầu', 'Thời gian thực hiện gói thầu'
    ]
    pre_1 = []
    for goithau_item in ttgoithau_list:
        if goithau_item in pre_key_1:
            pre_1.append(ttgoithau_dict[goithau_item])
        else:
            pre_1.append('No info')
    Ten_goi_thau = pre_1[0]
    Ben_moi_thau = pre_1[1]
    Nguon_ngan_sach = pre_1[2]
    Linh_vuc = pre_1[3]
    Hinh_thuc_chon_thau = pre_1[4]
    Loai_hop_dong = pre_1[5]
    Trongnuoc_quocte = pre_1[6]
    Phuong_thuc_chon_thau = pre_1[7]
    Thoi_gian_THHD = pre_1[8]
         #Thông tin dự thầu
    thongtingoithau = general_infor.contents[14]
    
    # Tìm tất cả các thẻ `div` có class `infomation__content__title`
    thongtingoithau_title = thongtingoithau.find_all('div', class_='infomation__content__title')
    
    # Khởi tạo danh sách để chứa tiêu đề và giá trị
    pre_key_2 = []
    pre_value_2 = []
    
    # Duyệt qua từng tiêu đề và lấy nội dung của nó cùng với giá trị tương ứng
    for tt_duthau in thongtingoithau_title:
        key_2 = tt_duthau.text.strip()  # Tiêu đề (key)
        
        # Tìm giá trị tương ứng sau thẻ `div` tiêu đề
        try:
            tt_duthau_value = tt_duthau.find_next_sibling('div')
            value_2 = tt_duthau_value.text.strip() if tt_duthau_value else 'No info'
        except (AttributeError, NameError):
            value_2 = 'No info'
        
        # Thêm tiêu đề và giá trị vào danh sách
        pre_key_2.append(key_2)
        pre_value_2.append(value_2)
    
    # Tạo từ điển từ hai danh sách
    ttduthau_dict = dict(zip(pre_key_2, pre_value_2))
    
    # Lấy thông tin "Số tiền bảo đảm dự thầu" và giữ lại định dạng số và đơn vị "VND"
    So_tien_dam_bao_du_thau_raw = ttduthau_dict.get('Số tiền bảo đảm dự thầu', 'No info')
    match = re.search(r'\d{1,3}(?:\.\d{3})* VND', So_tien_dam_bao_du_thau_raw)  # Tìm theo định dạng "x.xxx.xxx VND"
    So_tien_dam_bao_du_thau = match.group() if match else 'No info'
    
    # Lấy "Hình thức đảm bảo dự thầu" và chỉ lấy phần trước dấu "("
    Hinh_thuc_dam_bao_du_thau_raw = ttduthau_dict.get('Hình thức đảm bảo dự thầu', 'No info').strip()
    Hinh_thuc_dam_bao_du_thau = Hinh_thuc_dam_bao_du_thau_raw.split('(')[0].strip()

    # Loại công trình
    Loai_cong_trinh = ttduthau_dict.get('Loại công trình', 'No info').strip()
    # Thông tin quyêt định phê duyệt
    quyetdinhpheduyet = general_infor.contents[16]
    
    # Tìm tất cả các thẻ `div` có class `infomation__content__title`
    quyetdinhpheduyet_tile = quyetdinhpheduyet.find_all('div', class_='infomation__content__title')
    
    # Khởi tạo danh sách để chứa tiêu đề và giá trị
    pre_key_3 = []
    pre_value_3 = []
    
    # Duyệt qua từng tiêu đề và lấy nội dung của nó cùng với giá trị tương ứng
    for tt_quyetdinh in quyetdinhpheduyet_tile:
        key_3 = tt_quyetdinh.text.strip()  # Tiêu đề (key)
        
        # Tìm giá trị tương ứng sau thẻ `div` tiêu đề
        try:
            tt_quyetdinh_value = tt_quyetdinh.find_next_sibling('div')
            value_3 = tt_quyetdinh_value.text.strip() if tt_quyetdinh_value else 'No info'
        except (AttributeError, NameError):
            value_3 = 'No info'
        
        # Thêm tiêu đề và giá trị vào danh sách
        pre_key_3.append(key_3)
        pre_value_3.append(value_3)
    
    # Tạo từ điển từ hai danh sách
    ttquyetdinh_dict = dict(zip(pre_key_3, pre_value_3))
    
    # Danh sách các thông tin cần tìm
    ttquyetdinh_list = ['Số quyết định phê duyệt', 'Ngày phê duyệt', 'Cơ quan ban hành quyết định']
    pre_3 = []
    
    # Kiểm tra từng mục trong `ttquyetdinh_list` có tồn tại trong `pre_key_3` không
    for quyetdinh_item in ttquyetdinh_list:
        if quyetdinh_item in pre_key_3:
            pre_3.append(ttquyetdinh_dict[quyetdinh_item])
        else:
            pre_3.append('No info')
    
    # Gán giá trị cho các biến cuối cùng
    So_QĐPD = pre_3[0].strip()
    Ngay_ky_QĐPD = pre_3[1].strip()
    Co_quan_ban_hanh = pre_3[2].strip()


    # Lấy thông tin từ biên bản mời thầu:
    Gia_goi_thau = 0
    So_DN_du_thau = 0
    DN_du_thau_01 = 'Đang mở thầu'
    DN_du_thau_02 = 'Đang mở thầu'
    DN_du_thau_03 = 'Đang mở thầu'
    DN_du_thau_04 = 'Đang mở thầu'
    DN_du_thau_05 = 'Đang mở thầu'
    DN_du_thau_06 = 'Đang mở thầu'
    DN_du_thau_07 = 'Đang mở thầu'
    DN_du_thau_08 = 'Đang mở thầu'
    DN_du_thau_09 = 'Đang mở thầu'
    DN_du_thau_10 = 'Đang mở thầu'
    DN_du_thau_11 = 'Đang mở thầu'
    DN_du_thau_12 = 'Đang mở thầu'
    DN_du_thau_13 = 'Đang mở thầu'
    DN_du_thau_14 = 'Đang mở thầu'
    DN_du_thau_15 = 'Đang mở thầu'
    DN_du_thau_16 = 'Đang mở thầu'
    DN_du_thau_17 = 'Đang mở thầu'
    DN_du_thau_18 = 'Đang mở thầu'
    DN_du_thau_19 = 'Đang mở thầu'
    DN_du_thau_20 = 'Đang mở thầu'



    #Lấy thông tin kết quả lựa chọn nhà thầu
    DN_trung_thau_01 = 'Đang mở thầu'
    DN_trung_thau_02 = 'Đang mở thầu'
    DN_trung_thau_03 = 'Đang mở thầu'
    DN_trung_thau_04 = 'Đang mở thầu'
    DN_trung_thau_05 = 'Đang mở thầu'
    DN_trung_thau_06 = 'Đang mở thầu'
    DN_trung_thau_07 = 'Đang mở thầu'
    DN_trung_thau_08 = 'Đang mở thầu'
    DN_trung_thau_09 = 'Đang mở thầu'
    DN_trung_thau_10 = 'Đang mở thầu'
    DN_trung_thau_11 = 'Đang mở thầu'
    DN_trung_thau_12 = 'Đang mở thầu'
    DN_trung_thau_13 = 'Đang mở thầu'
    DN_trung_thau_14 = 'Đang mở thầu'
    DN_trung_thau_15 = 'Đang mở thầu'
    DN_trung_thau_16 = 'Đang mở thầu'
    DN_trung_thau_17 = 'Đang mở thầu'
    DN_trung_thau_18 = 'Đang mở thầu'
    DN_trung_thau_19 = 'Đang mở thầu'
    DN_trung_thau_20 = 'Đang mở thầu'
    Ngay_ky_HD = '01/01/2999'
    Ngay_ky_HD = datetime.strptime(Ngay_ky_HD, '%d/%m/%Y')
    Trang_thai_goi_thau = 'Đang mở thầu'
    now = datetime.now()
    Ngay_keo_du_lieu = now.strftime("%d/%m/%Y %H:%M:%S")
    Ngay_keo_du_lieu = datetime.strptime(Ngay_keo_du_lieu, "%d/%m/%Y %H:%M:%S")
    Sql_2 = "Insert into SME..TAIMX_SRC_Du_LIEU_GOI_THAU_DETAIL (Ma_TBMT,Ten_goi_thau,Ben_moi_thau,Nguon_ngan_sach,Linh_vuc,Hinh_thuc_chon_thau,Loai_hop_dong,Trongnuoc_quocte,Phuong_thuc_chon_thau,Thoi_gian_THHD,So_tien_dam_bao_du_thau,Hinh_thuc_dam_bao_du_thau,Loai_cong_trinh,So_QĐPD,Ngay_ky_QĐPD,Co_quan_ban_hanh,Gia_goi_thau,So_DN_du_thau,DN_du_thau_01,DN_du_thau_02,DN_du_thau_03,DN_du_thau_04,DN_du_thau_05,DN_du_thau_06,DN_du_thau_07,DN_du_thau_08,DN_du_thau_09,DN_du_thau_10,DN_du_thau_11,DN_du_thau_12,DN_du_thau_13,DN_du_thau_14,DN_du_thau_15,DN_du_thau_16,DN_du_thau_17,DN_du_thau_18,DN_du_thau_19,DN_du_thau_20,DN_trung_thau_01,DN_trung_thau_02,DN_trung_thau_03,DN_trung_thau_04,DN_trung_thau_05,DN_trung_thau_06,DN_trung_thau_07,DN_trung_thau_08,DN_trung_thau_09,DN_trung_thau_10,DN_trung_thau_11,DN_trung_thau_12,DN_trung_thau_13,DN_trung_thau_14,DN_trung_thau_15,DN_trung_thau_16,DN_trung_thau_17,DN_trung_thau_18,DN_trung_thau_19,DN_trung_thau_20,Ngay_ky_HD,Trang_thai_goi_thau,Ngay_keo_du_lieu) Values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
    cursor_3.execute(Sql_2,(Ma_TBMT,Ten_goi_thau,Ben_moi_thau,Nguon_ngan_sach,Linh_vuc,Hinh_thuc_chon_thau,Loai_hop_dong,Trongnuoc_quocte,Phuong_thuc_chon_thau,Thoi_gian_THHD,So_tien_dam_bao_du_thau,Hinh_thuc_dam_bao_du_thau,Loai_cong_trinh,So_QĐPD,Ngay_ky_QĐPD,Co_quan_ban_hanh,Gia_goi_thau,So_DN_du_thau,DN_du_thau_01,DN_du_thau_02,DN_du_thau_03,DN_du_thau_04,DN_du_thau_05,DN_du_thau_06,DN_du_thau_07,DN_du_thau_08,DN_du_thau_09,DN_du_thau_10,DN_du_thau_11,DN_du_thau_12,DN_du_thau_13,DN_du_thau_14,DN_du_thau_15,DN_du_thau_16,DN_du_thau_17,DN_du_thau_18,DN_du_thau_19,DN_du_thau_20,DN_trung_thau_01,DN_trung_thau_02,DN_trung_thau_03,DN_trung_thau_04,DN_trung_thau_05,DN_trung_thau_06,DN_trung_thau_07,DN_trung_thau_08,DN_trung_thau_09,DN_trung_thau_10,DN_trung_thau_11,DN_trung_thau_12,DN_trung_thau_13,DN_trung_thau_14,DN_trung_thau_15,DN_trung_thau_16,DN_trung_thau_17,DN_trung_thau_18,DN_trung_thau_19,DN_trung_thau_20,Ngay_ky_HD,Trang_thai_goi_thau,Ngay_keo_du_lieu))
    connection_3.commit()
    Sql_update_sentence = "Update SME..TAIMX_DIM_DU_LIEU_GOI_THAU_OVERVIEW Set Remark = '1' where Ma_TBMT ='"+ Ma_TBMT +"'"
    cursor_3.execute(Sql_update_sentence)
    connection_3.commit()
    print('Written successfully for the contract no:',Ma_TBMT)
    print('-----------------------------------', j)
    browser.close()

In [ ]:
# import random
# import time
# from selenium import webdriver
# from selenium.webdriver.edge.service import Service
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.by import By
# from bs4 import BeautifulSoup

# # Danh sách proxy IPs


# # Kiểm tra điều kiện "ready" và danh sách dữ liệu
# if ready == 'Yes' and len(query_result) > 0:
#     exclude_lst = ["X", "X", "X", "X", "X", "X", "X", "X", "X"]
#     t = 1

#     for j in range(len(query_result)):
#         # Lọc danh sách proxy chưa bị loại trừ
#         preIP_lst = [item for item in ID_fix_list if item not in exclude_lst]
#         link = query_result.iloc[j, 3]
#         preIP = random.choice(preIP_lst)
#         proxy = preIP + str(random.randint(1, 200)) + ":80"
#         print("Using proxy:", proxy)

#         # Cập nhật danh sách exclude_lst
#         exclude_lst[t % 9 - 1] = preIP
#         t += 1

#         # Cấu hình trình duyệt Edge với proxy
#         edge_options = webdriver.EdgeOptions()
#         edge_options.add_argument(f"--proxy-server=http://{proxy}")
#         driver_path = 'D:/Driver Egde/msedgedriver.exe'
#         service = Service(executable_path=driver_path)
#         browser = webdriver.Edge(service=service, options=edge_options)
        
#         try:
#             browser.get(link)
#             soup = BeautifulSoup(browser.page_source, 'html.parser')
#             span_tag = soup.find('span', {
#                 'jsselect': 'heading',
#                 'jstcache': '14',
#                 'jsvalues': '.innerHTML:msg'
#             })

#             # Kiểm tra nếu thẻ <span> chứa "Hmmm… can't reach this page"
#             if span_tag and "Hmmm… can't reach this page" in span_tag.text:
#                 print("Page not reachable, continuing with next proxy...")
#                 browser.quit()
#                 continue  # Chuyển sang vòng lặp tiếp theo

#             # Thực hiện các hành động khi trang có thể truy cập
#             try:
#                 WebDriverWait(browser, 3).until(EC.element_to_be_clickable((By.XPATH, '//button[@id="details-button"]')))
#                 advanced = browser.find_element(By.XPATH, '//button[@id="details-button"]')
#                 advanced.click()
#                 proceed_link = browser.find_element(By.XPATH, '//a[@id="proceed-link"]')
#                 proceed_link.click()
#                 time.sleep(random.randint(2, 4))
#             except:
#                 print("Details button not found or could not proceed.")

#             # Xử lý các trường hợp của Loai_goi_thau
#             if Loai_goi_thau == 'Đã trúng thầu':
#                 try:
#                     Da_trung_thau()
#                 except:
#                     print('Skip MTBMT: ', query_result.iloc[j, 0])
#             elif Loai_goi_thau == 'Đang xét thầu':
#                 try:
#                     Dang_xet_thau()
#                 except:
#                     print('Skip MTBMT: ', query_result.iloc[j, 0])
#             elif Loai_goi_thau == 'Đang mở thầu':
#                 try:
#                     Dang_mo_thau()
#                 except:
#                     print('Skip MTBMT: ', query_result.iloc[j, 0])

#             browser.quit()
#         except Exception as e:
#             print("Error:", e)
#             browser.quit()

#     print('DONE!!!!!!')
# else:
#     if len(query_result) <= 0:
#         print('OHH! We found no data in the table TAIMX_DIM_DU_LIEU_GOI_THAU_OVERVIEW, please recheck!')
#     else:
#         print('SORRY YOU ARE NOT READY TO RUN THE CODE, SEE YOU NEXT TIME!!!')


In [ ]:
# ID_fix_list = ["172.217.6."
#                   ,"211.45.026."
#                   ,"210.245.126."
#                   ,"91.15.73."
#                   ,"50.50.50."
#                   ,"117.58.6."
#                   ,"116.100.6."
#                   ,"42.116.6."
#                   ,"117.248.6."
#                   ,"113.161.118."
#                   ,"117.20.121."
#                   ,"202.96.12."
#                   ,"91.211.245."
#                   ,"176.31.200."
#                   ,"173.192.21."
#                   ,"188.100.212."
#                   ,"81.171.24."
#                   ,'145.239.85.'
#                   ,'46.4.96.'
#                   ,'47.91.88.'
#                   ,'45.77.56.'
#                   ,'82.196.11.'
#                   ,'51.254.69.'
#                   ,'178.62.193.'
#                   ,'188.226.141.'
#                   ,'217.23.6.'
#                   ,'185.153.198.'
#                   ,'81.171.24.'
#                   ,'5.189.224.'
#                   ,'108.61.175.'
#                   ,'176.31.200.'
#                   ,'83.77.118.'
#                   ,'173.192.21.'
#                   ,'163.172.182.'
#                   ,'163.172.168.'
#                   ,'164.68.105.'
#                   ,'5.199.171.'
#                   ,'93.171.164.'
#                   ,'212.112.97.'
#                   ,'51.68.207.'
#                   ,'91.211.245.'
#                   ,'84.201.254.'
#                   ,'95.156.82.'
#                   ,'185.118.141.'
#                   ,'164.68.98.'
#                   ,'217.113.122.'
#                   ,'188.100.212.'
#                   ,'83.79.50.'
#                 ]
# preIP = random.choice(ID_fix_list)
# proxy = preIP + str(randint(1,200)) + ":80"
# #print(proxy)
# option = webdriver.EdgeOptions()
# option.add_argument("--proxy-server=%s" % proxy)
# # option.add_argument("--headless") 
# # option.add_argument("--disable-gpu")
# driver_path = 'D:/Driver Egde/msedgedriver.exe'
# service = Service(executable_path=driver_path)
# browser = webdriver.Edge(service=service,options = option)
# browser.get("https://muasamcong.mpi.gov.vn/web/guest/contractor-selection?p_p_id=egpportalcontractorselectionv2_WAR_egpportalcontractorselectionv2&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_egpportalcontractorselectionv2_WAR_egpportalcontractorselectionv2_render=detail-v2&type=es-notify-contractor&stepCode=notify-contractor-step-1-tbmt&id=ad574123-584f-46ec-b03f-e93e988192b4&notifyId=ad574123-584f-46ec-b03f-e93e988192b4&inputResultId=undefined&bidOpenId=undefined&techReqId=undefined&bidPreNotifyResultId=undefined&bidPreOpenId=undefined&processApply=KHAC&bidMode=1_MTHS&notifyNo=IB2400401470&planNo=PL2400226516&pno=undefined&step=tbmt&isInternet=0&caseKHKQ=undefined")

In [ ]:
# WebDriverWait(browser, 60).until(EC.presence_of_element_located((By.ID, 'info-general')))
# page_source = BeautifulSoup(browser.page_source,'html.parser')
# general_infor = page_source.find('div',id='info-general')
# print(general_infor)

In [ ]:
# Da_trung_thau()

In [ ]:
# thongtingoithau = general_infor.contents[8]

In [ ]:
# thongtingoithau.find_all('div',class_='infomation__content__title')

In [ ]:
# from bs4 import Comment

# # Lấy phần tử thứ 8 từ general_infor.contents
# thongtingoithau = general_infor.contents[8]

# # Kiểm tra xem phần tử thongtingoithau có tồn tại và không phải là bình luận HTML
# if thongtingoithau and not isinstance(thongtingoithau, Comment):
#     print("Phần tử thongtingoithau tồn tại và không phải là một bình luận HTML.")
    
#     # Tìm tất cả các thẻ <div> với class 'infomation__content__title'
#     thongtingoithau_title = thongtingoithau.find_all('div', class_='infomation__content__title')

#     # Kiểm tra xem có tìm thấy thẻ nào không
#     if thongtingoithau_title:
#         print(f"Đã tìm thấy {len(thongtingoithau_title)} thẻ <div> với class 'infomation__content__title'.")
#         for title in thongtingoithau_title:
#             print(f"Nội dung của thẻ <div>: {title.text.strip()}")
#     else:
#         print("Không tìm thấy thẻ <div> nào với class 'infomation__content__title'.")
# else:
#     print("Phần tử thongtingoithau không tồn tại hoặc là một bình luận HTML.")



In [ ]:
# time.sleep(5)
# Dang_xet_thau()

In [ ]:
# import random
# import time
# from selenium import webdriver
# from selenium.webdriver.edge.service import Service
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.by import By

# ID_fix_list = ["172.217.6."
#                   ,"211.45.026."
#                   ,"210.245.126."
#                   ,"91.15.73."
#                   ,"50.50.50."
#                   ,"117.58.6."
#                   ,"116.100.6."
#                   ,"42.116.6."
#                   ,"117.248.6."
#                   ,"113.161.118."
#                   ,"117.20.121."
#                   ,"202.96.12."
#                   ,"91.211.245."
#                   ,"176.31.200."
#                   ,"173.192.21."
#                   ,"188.100.212."
#                   ,"81.171.24."
#                   ,'145.239.85.'
#                   ,'46.4.96.'
#                   ,'47.91.88.'
#                   ,'45.77.56.'
#                   ,'82.196.11.'
#                   ,'51.254.69.'
#                   ,'178.62.193.'
#                   ,'188.226.141.'
#                   ,'217.23.6.'
#                   ,'185.153.198.'
#                   ,'81.171.24.'
#                   ,'5.189.224.'
#                   ,'108.61.175.'
#                   ,'176.31.200.'
#                   ,'83.77.118.'
#                   ,'173.192.21.'
#                   ,'163.172.182.'
#                   ,'163.172.168.'
#                   ,'164.68.105.'
#                   ,'5.199.171.'
#                   ,'93.171.164.'
#                   ,'212.112.97.'
#                   ,'51.68.207.'
#                   ,'91.211.245.'
#                   ,'84.201.254.'
#                   ,'95.156.82.'
#                   ,'185.118.141.'
#                   ,'164.68.98.'
#                   ,'217.113.122.'
#                   ,'188.100.212.'
#                   ,'83.79.50.'
#                 ]
# # Kiểm tra điều kiện sẵn sàng
# if ready == 'Yes' and len(query_result) > 0:
#     exclude_lst = ["X"] * 9  # Danh sách các IP cần loại trừ
#     t = 1

#     for j in range(len(query_result)):
#         # Lọc danh sách các IP không nằm trong exclude_lst
#         preIP_lst = [item for item in ID_fix_list if item not in exclude_lst]

#         if not preIP_lst:
#             print("Hết proxy khả dụng. Vui lòng kiểm tra danh sách proxy của bạn.")
#             break

#         link = query_result.iloc[j, 3]
#         preIP = random.choice(preIP_lst)
#         proxy = preIP + str(random.randint(1, 200)) + ":80"
#         print("Using proxy:", proxy)

#         exclude_lst[t % 9 - 1] = preIP  # Cập nhật danh sách loại trừ
#         t += 1

#         # Cấu hình trình duyệt Edge
#         edge_options = webdriver.EdgeOptions()
#         # edge_options.add_argument("--headless")  # Bỏ chú thích nếu cần chế độ không giao diện
#         # edge_options.add_argument("--disable-gpu")
#         edge_options.add_argument(f"--proxy-server=http://{proxy}")
#         driver_path = 'D:/Driver Egde/msedgedriver.exe'
#         service = Service(executable_path=driver_path)

#         try:
#             browser = webdriver.Edge(service=service, options=edge_options)
#             browser.get(link)  # Thử truy cập liên kết

#             # Chờ phần tử xuất hiện và thực hiện các thao tác
#             try:
#                 WebDriverWait(browser, 3).until(
#                     EC.element_to_be_clickable((By.XPATH, '//button[@id="details-button"]'))
#                 )
#                 advanced = browser.find_element(By.XPATH, '//button[@id="details-button"]')
#                 advanced.click()
#                 proceed_link = browser.find_element(By.XPATH, '//a[@id="proceed-link"]')
#                 proceed_link.click()
#                 time.sleep(random.randint(2, 4))
#                             # Xử lý theo loại gói thầu
#                 if Loai_goi_thau == 'Đã trúng thầu':
#                     try:
#                         Da_trung_thau()
#                     except Exception as e:
#                         print('Skip MTBMT:', query_result.iloc[j, 0], "Error:", e)
#                 elif Loai_goi_thau == 'Đang xét thầu':
#                     try:
#                         Dang_xet_thau()
#                     except Exception as e:
#                         print('Skip MTBMT:', query_result.iloc[j, 0], "Error:", e)
#                 elif Loai_goi_thau == 'Đang mở thầu':
#                     try:
#                         Dang_mo_thau()
#                     except Exception as e:
#                         print('Skip MTBMT:', query_result.iloc[j, 0], "Error:", e)
#                 browser.quit()
#         except Exception as e:
#             except Exception as e:
#                 print("Details button not found or could not proceed. Error:", e)


#             print(f"Proxy {proxy} không hoạt động. Lỗi: {e}")
#             preIP_lst.remove(preIP)  # Loại bỏ proxy không hoạt động khỏi danh sách
#             browser.quit()

#     print('DONE!!!!!!')
# else:
#     if len(query_result) <= 0:
#         print('OHH! We found no data in the table TAIMX_DIM_DU_LIEU_GOI_THAU_OVERVIEW, please recheck!')
#     else:
#         print('SORRY YOU ARE NOT READY TO RUN THE CODE, SEE YOU NEXT TIME!!!')


In [ ]:
import random
import time
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
 
ID_fix_list = ["172.217.6."
                  ,"211.45.026."
                  ,"210.245.126."
                  ,"91.15.73."
                  ,"50.50.50."
                  ,"117.58.6."
                  ,"116.100.6."
                  ,"42.116.6."
                  ,"117.248.6."
                  ,"113.161.118."
                  ,"117.20.121."
                  ,"202.96.12."
                  ,"91.211.245."
                  ,"176.31.200."
                  ,"173.192.21."
                  ,"188.100.212."
                  ,"81.171.24."
                  ,'145.239.85.'
                  ,'46.4.96.'
                  ,'47.91.88.'
                  ,'45.77.56.'
                  ,'82.196.11.'
                  ,'51.254.69.'
                  ,'178.62.193.'
                  ,'188.226.141.'
                  ,'217.23.6.'
                  ,'185.153.198.'
                  ,'81.171.24.'
                  ,'5.189.224.'
                  ,'108.61.175.'
                  ,'176.31.200.'
                  ,'83.77.118.'
                  ,'173.192.21.'
                  ,'163.172.182.'
                  ,'163.172.168.'
                  ,'164.68.105.'
                  ,'5.199.171.'
                  ,'93.171.164.'
                  ,'212.112.97.'
                  ,'51.68.207.'
                  ,'91.211.245.'
                  ,'84.201.254.'
                  ,'95.156.82.'
                  ,'185.118.141.'
                  ,'164.68.98.'
                  ,'217.113.122.'
                  ,'188.100.212.'
                  ,'83.79.50.'
                ]
 
if ready.lower() == 'yes' and len(query_result) > 0:
    exclude_lst = ["X", "X", "X", "X", "X", "X", "X", "X", "X"]
    t = 1
 
    for j in range(len(query_result)):
        preIP_lst = [item for item in ID_fix_list if item not in exclude_lst]
        link = query_result.iloc[j, 3]
        preIP = random.choice(preIP_lst)
        proxy = preIP + str(random.randint(1, 200)) + ":80"
        print("Using proxy:", proxy)
 
  
        exclude_lst[t % 9 - 1] = preIP
        t += 1
 
        edge_options = webdriver.EdgeOptions()
#         edge_options.add_argument("--headless") 
#         edge_options.add_argument("--disable-gpu")
        edge_options.add_argument(f"--proxy-server=http://{proxy}")
        driver_path = 'D:/Driver Egde/msedgedriver.exe'
        service = Service(executable_path=driver_path)
        try:
            browser = webdriver.Edge(service=service, options=edge_options)
            browser.get(link)
#             soup = BeautifulSoup(browser.page_source, 'html.parser')
#             span_tag = soup.find('span', {
#                 'jsselect': 'heading',
#                 'jstcache': '14',
#                 'jsvalues': '.innerHTML:msg'
#             })
 
        
#             if span_tag and "Hmmm… can't reach this page" in span_tag.text:
#                 print("Page not reachable, continuing with next proxy...")
#                 browser.quit()
#                 continue
 
           
            try:
                WebDriverWait(browser, 3).until(EC.element_to_be_clickable((By.XPATH, '//button[@id="details-button"]')))
                advanced = browser.find_element(By.XPATH, '//button[@id="details-button"]')
                advanced.click()
                proceed_link = browser.find_element(By.XPATH, '//a[@id="proceed-link"]')
                proceed_link.click()
                time.sleep(random.randint(2, 4))
            except:
                print("-----")
            if Loai_goi_thau == 'Đã trúng thầu':
                try:
                    Da_trung_thau()
                except:
                    print('Skip MTBMT: ', query_result.iloc[j, 0])
            elif Loai_goi_thau == 'Đang xét thầu':
                try:
                    Dang_xet_thau()
                except:
                    print('Skip MTBMT: ', query_result.iloc[j, 0])
            elif Loai_goi_thau == 'Đang mở thầu':
                try:
                    Dang_mo_thau()
                except:
                    print('Skip MTBMT: ', query_result.iloc[j, 0])
 
            browser.quit()
        except Exception as e:
            print(f"Proxy {proxy} không hoạt động. Lỗi: {e}")
            preIP_lst.remove(preIP) 
            browser.quit()
            continue
    print('DONE!!!!!!')
else:
    if len(query_result) <= 0:
        print('OHH! We found no data in the table TAIMX_DIM_DU_LIEU_GOI_THAU_OVERVIEW, please recheck!')
    else:
        print('SORRY YOU ARE NOT READY TO RUN THE CODE, SEE YOU NEXT TIME!!!')